In [1]:
import igibson.object_states as object_states
from igibson.tasks.behavior_task import BehaviorTask
from igibson.utils.ig_logging import IGLogReader
from igibson.utils.utils import parse_config
import os
import igibson
from igibson.envs.igibson_env import iGibsonEnv
from igibson.transition_model_v3.eval_env import EvalEnv
from igibson.transition_model_v3.eval_env import EvalActions
import igibson.object_states as object_states
from tqdm import tqdm
import json
import openai
import time


demo_to_conds_path = "/Users/bryan/Desktop/wkdir/behavior-vllm-eval/igibson/evaluation/goal_interpretation/assets/all_conditions.json"
demo_to_objs_path = "/Users/bryan/Desktop/wkdir/behavior-vllm-eval/igibson/evaluation/goal_interpretation/assets/all_objects.json"
demo_names_path = "/Users/bryan/Desktop/wkdir/behavior-vllm-eval/igibson/evaluation/goal_interpretation/assets/100_selected_demos.txt"
task_to_instructions_path = "/Users/bryan/Desktop/wkdir/behavior-vllm-eval/igibson/evaluation/goal_interpretation/assets/instructions_by_activity_name.json"
prompt_path = "/Users/bryan/Desktop/wkdir/behavior-vllm-eval/igibson/evaluation/goal_interpretation/prompts/behavior_goal_interpretation.txt"
task_to_demo_path = "/Users/bryan/Desktop/wkdir/behavior-vllm-eval/igibson/evaluation/goal_interpretation/assets/task_to_demo.json"

with open(demo_to_conds_path, 'r') as json_file:
    demo_to_conds = json.load(json_file)

with open(demo_to_objs_path, 'r') as json_file:
    demo_to_objs = json.load(json_file)

with open(task_to_instructions_path, 'r') as json_file:
    task_to_instructions = json.load(json_file)
    
with open(task_to_demo_path, 'r') as json_file:
    task_to_demos = json.load(json_file)

with open(demo_names_path, 'r') as file:
    demo_names = file.read().splitlines()


 _   _____  _  _
(_) / ____|(_)| |
 _ | |  __  _ | |__   ___   ___   _ __
| || | |_ || || '_ \ / __| / _ \ | '_ \
| || |__| || || |_) |\__ \| (_) || | | |
|_| \_____||_||_.__/ |___/ \___/ |_| |_|



WARN: Darwin does not support optimized renderer, automatically disabling
WARN: Darwin does not support optimized renderer, automatically disabling
WARN: Darwin does not support optimized renderer, automatically disabling
WARN: Darwin does not support optimized renderer, automatically disabling
WARN: Darwin does not support optimized renderer, automatically disabling
WARN: Darwin does not support optimized renderer, automatically disabling
torch is not available, falling back to rendering to memory (instead of tensor)


In [2]:
gpt35_results_path = "/Users/bryan/Desktop/wkdir/behavior-vllm-eval/igibson/evaluation/goal_interpretation/assets/gpt35_goal_interpretation.json"
gpt4_results_path = "/Users/bryan/Desktop/wkdir/behavior-vllm-eval/igibson/evaluation/goal_interpretation/assets/gpt4_goal_interpretation.json"

with open(gpt35_results_path, 'r') as json_file:
    gpt35_results = json.load(json_file)

with open(gpt4_results_path, 'r') as json_file:
    gpt4_results = json.load(json_file)

In [3]:
for key, value in demo_to_conds.items():
    demo_name = key
    goal_conds = value['goal_conditions']
    
    gpt4_pred = gpt4_results[demo_name]
    gpt35_pred = gpt35_results[demo_name]
    
    print("goal_conds: ", goal_conds)
    print("gpt4_pred: ", gpt4_pred)
    





goal_conds:  {'1': [[['not', ['stained', 'bathtub.n.01_1']]]]}
gpt4_pred:  {'node goals': [['not', ['Stained', 'bathtub.n.01_1']]], 'edge goals': []}
goal_conds:  {'1': [[['inside', 'plate.n.04_1', 'cabinet.n.01_1'], ['inside', 'plate.n.04_2', 'cabinet.n.01_1'], ['inside', 'plate.n.04_3', 'cabinet.n.01_1'], ['inside', 'plate.n.04_4', 'cabinet.n.01_1'], ['inside', 'plate.n.04_5', 'cabinet.n.01_1'], ['inside', 'plate.n.04_6', 'cabinet.n.01_1'], ['inside', 'plate.n.04_7', 'cabinet.n.01_1'], ['inside', 'plate.n.04_8', 'cabinet.n.01_1']]]}
gpt4_pred:  {'node goals': [], 'edge goals': [['inside', 'plate.n.04_1', 'cabinet1.n.01_1'], ['inside', 'plate.n.04_2', 'cabinet1.n.01_1'], ['inside', 'plate.n.04_3', 'cabinet1.n.01_1'], ['inside', 'plate.n.04_4', 'cabinet1.n.01_1'], ['inside', 'plate.n.04_5', 'cabinet1.n.01_1'], ['inside', 'plate.n.04_6', 'cabinet1.n.01_1'], ['inside', 'plate.n.04_7', 'cabinet1.n.01_1'], ['inside', 'plate.n.04_8', 'cabinet1.n.01_1']]}
goal_conds:  {'1': [[['not', ['stain

In [5]:
import json
from collections import defaultdict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def flatten_goals(goal_data):
    """Flatten goal data into a single list of conditions."""
    return [condition for goal_type in goal_data.values() for condition in goal_type]

def normalize_condition(condition):
    """Normalize conditions to ensure uniform comparison."""
    if isinstance(condition[1], list):
        return (condition[0].lower(),) + tuple(sorted(normalize_condition(subcond) for subcond in condition[1]))
    return tuple(map(str.lower, condition))

def evaluate_goals(predicted_goals, ground_truth_goals):
    """Evaluate predicted goals against ground truth goals."""
    # Flatten and normalize the predicted goals
    predicted_conditions = set(normalize_condition(cond) for cond in flatten_goals(predicted_goals))
    
    # Process ground truth to accommodate alternative satisfactions
    ground_truth_conditions = []
    for alternatives in ground_truth_goals.values():
        for option in alternatives:
            option_conditions = set(normalize_condition(cond) for cond in option)
            ground_truth_conditions.append(option_conditions)
    
    # Determine matches
    matches = defaultdict(bool)
    for i, option_conditions in enumerate(ground_truth_conditions):
        if option_conditions.issubset(predicted_conditions):
            matches[i] = True
    
    # Compute evaluation metrics
    true_positives = sum(matches.values())
    false_positives = len(predicted_conditions) - true_positives
    false_negatives = len(ground_truth_conditions) - true_positives
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Calculate overall accuracy
    accuracy = true_positives / len(ground_truth_conditions) if ground_truth_conditions else 0
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }

# Example usage
predicted_goals_json = '{"node goals": [["not", ["Stained", "pan.n.01_1"]], ["not", ["Stained", "pan.n.01_2"]]], "edge goals": [["inside", "pan.n.01_1", "cabinet.n.01_1"]]}'
ground_truth_goals_json = '{"1": [[["not", ["stained", "pan.n.01_1"]], ["inside", "pan.n.01_1", "cabinet.n.01_1"]]], "2": [[["not", ["stained", "kettle.n.01_1"]], ["inside", "kettle.n.01_1", "cabinet.n.01_1"]]]}'

predicted_goals = json.loads(predicted_goals_json)
ground_truth_goals = json.loads(ground_truth_goals_json)

results = evaluate_goals(predicted_goals, ground_truth_goals)
print(results)


defaultdict(<class 'bool'>, {0: True})
{'accuracy': 0.5, 'precision': 0.3333333333333333, 'recall': 0.5, 'f1_score': 0.4}


In [16]:
import json
from collections import defaultdict

def flatten_goals(goal_data):
    """Flatten goal data into a single list of conditions."""
    return [condition for goal_type in goal_data.values() for condition in goal_type]

def normalize_condition(condition):
    """Normalize conditions to ensure uniform comparison."""
    if isinstance(condition[1], list):
        return (condition[0].lower(),) + tuple(sorted(normalize_condition(subcond) for subcond in condition[1]))
    return tuple(map(str.lower, condition))

def evaluate_goals(predicted_goals, ground_truth_goals):
    """Evaluate predicted goals against ground truth goals."""
    # Flatten and normalize the predicted goals
    predicted_conditions = set(normalize_condition(cond) for cond in flatten_goals(predicted_goals))
    
    print("predicted_conditions", predicted_conditions)
    
    # Process ground truth to accommodate alternative satisfactions
    ground_truth_conditions = []
    all_conditions = set()
    for alternatives in ground_truth_goals.values():
        for option in alternatives:
            option_conditions = set(normalize_condition(cond) for cond in option)
            ground_truth_conditions.append(option_conditions)
            all_conditions.update(option_conditions)

    # Determine matches
    matches = defaultdict(bool)
    matched_conditions = set()
    for option_conditions in ground_truth_conditions:
        if option_conditions.issubset(predicted_conditions):
            matches[tuple(option_conditions)] = True
            matched_conditions.update(option_conditions)
    
    unmatched_conditions = all_conditions - matched_conditions

    # Compute evaluation metrics
    true_positives = len(matched_conditions)
    false_positives = len(predicted_conditions - matched_conditions)
    false_negatives = len(unmatched_conditions)
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'accuracy': true_positives / len(all_conditions) if all_conditions else 0,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'matched_conditions': list(map(list, matched_conditions)),
        'unmatched_conditions': list(map(list, unmatched_conditions))
    }


TypeError: unhashable type: 'list'

In [25]:
import json
from collections import defaultdict

def flatten_goals(goal_data):
    """Flatten goal data into a single list of conditions."""
    return [condition for goal_type in goal_data.values() for condition in goal_type]

def evaluate_goals(predicted_goals, ground_truth_goals):
    """Evaluate predicted goals against ground truth goals."""
    # Flatten and normalize the predicted goals
    predicted_conditions = [cond for cond in flatten_goals(predicted_goals)]
    
    print("predicted_conditions", predicted_conditions)
    
    # Process ground truth to accommodate alternative satisfactions
    ground_truth_conditions = []
    all_conditions = []
    for alternatives in ground_truth_goals.values():
        for option in alternatives:
            option_conditions = [cond for cond in option]
            ground_truth_conditions.append(option_conditions)
            all_conditions.extend(option_conditions)

    # Determine matches
    matches = defaultdict(bool)
    matched_conditions = []
    for option_conditions in ground_truth_conditions:
        if option_conditions.issubset(predicted_conditions):
            matches[tuple(option_conditions)] = True
            matched_conditions.extend(option_conditions)
    
    unmatched_conditions = all_conditions - matched_conditions

    # Compute evaluation metrics
    true_positives = len(matched_conditions)
    false_positives = len(predicted_conditions - matched_conditions)
    false_negatives = len(unmatched_conditions)
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'accuracy': true_positives / len(all_conditions) if all_conditions else 0,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'matched_conditions': list(map(list, matched_conditions)),
        'unmatched_conditions': list(map(list, unmatched_conditions))
    }

In [29]:
import json
from collections import defaultdict

def flatten_goals(goal_data):
    """Flatten goal data into a single list of conditions."""
    return [condition for goal_type in goal_data.values() for condition in goal_type]

def check_satisfaction(predicted_conditions, ground_truth_conditions):
    satisfied_conditions = []
    unsatisfied_conditions = []
    for condition in ground_truth_conditions:
        if condition in predicted_conditions:
            satisfied_conditions.append(condition)
        else:
            unsatisfied_conditions.append(condition)
    return satisfied_conditions, unsatisfied_conditions


def evaluate_goals(predicted_goals, ground_truth_goals):
    """Evaluate predicted goals against ground truth goals."""
    # Flatten the predicted goals
    predicted_conditions = flatten_goals(predicted_goals)
    
    
    satisfied_conditions = []
    unsatisfied_conditions = []
    
    # check each goal in ground_truth_goals
    for key, value in ground_truth_goals.items():
        # if there are multiple ways to satisfy the goal
        for option in value:
            if all(cond in predicted_conditions for cond in option):
                satisfied_conditions.extend(option)
                break
        unsatisfied_conditions.extend(value[0])
                                   
            
    
    
    
    
    
    
    # Determine matches
    matches = defaultdict(bool)
    matched_conditions = []
    for option_conditions in ground_truth_conditions:
        if all(cond in predicted_conditions for cond in option_conditions):
            matches[tuple(option_conditions)] = True
            matched_conditions.extend(option_conditions)
    
    unmatched_conditions = [cond for cond in all_conditions if cond not in matched_conditions]

    # Compute evaluation metrics
    true_positives = len(matched_conditions)
    false_positives = len([cond for cond in predicted_conditions if cond not in matched_conditions])
    false_negatives = len(unmatched_conditions)
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'accuracy': true_positives / len(all_conditions) if all_conditions else 0,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'matched_conditions': matched_conditions,
        'unmatched_conditions': unmatched_conditions
    }

# Example usage
predicted_goals_json = '{"node goals": [["not", ["stained", "pan.n.01_1"]], ["not", ["stained", "pan.n.01_2"]]], "edge goals": [["inside", "pan.n.01_1", "cabinet.n.01_1"]]}'
ground_truth_goals_json = '{"1": [[["not", ["stained", "pan.n.01_1"]], ["inside", "pan.n.01_1", "cabinet.n.01_1"]]], "2": [[["not", ["stained", "kettle.n.01_1"]], ["inside", "kettle.n.01_1", "cabinet.n.01_1"]]]}'

predicted_goals = json.loads(predicted_goals_json)
ground_truth_goals = json.loads(ground_truth_goals_json)

results = evaluate_goals(predicted_goals, ground_truth_goals)
print("Results:", results)


ground_truth_conditions: [[['not', ['stained', 'pan.n.01_1']], ['inside', 'pan.n.01_1', 'cabinet.n.01_1']], [['not', ['stained', 'kettle.n.01_1']], ['inside', 'kettle.n.01_1', 'cabinet.n.01_1']]]
all_conditions: [['not', ['stained', 'pan.n.01_1']], ['inside', 'pan.n.01_1', 'cabinet.n.01_1'], ['not', ['stained', 'kettle.n.01_1']], ['inside', 'kettle.n.01_1', 'cabinet.n.01_1']]


TypeError: unhashable type: 'list'

In [30]:
demo = "assembling_gift_baskets_0_Beechwood_0_int_0_2021-10-26_12-46-37"

goal_conds = demo_to_conds[demo]['goal_conditions']
gpt4_pred = gpt4_results[demo]

results = evaluate_goals(gpt4_pred, goal_conds)

ground_truth_conditions: [[['inside', 'candle.n.01_1', 'basket.n.01_1'], ['inside', 'candle.n.01_2', 'basket.n.01_2'], ['inside', 'candle.n.01_3', 'basket.n.01_3'], ['inside', 'candle.n.01_4', 'basket.n.01_4']], [['inside', 'candle.n.01_1', 'basket.n.01_1'], ['inside', 'candle.n.01_2', 'basket.n.01_2'], ['inside', 'candle.n.01_3', 'basket.n.01_4'], ['inside', 'candle.n.01_4', 'basket.n.01_3']], [['inside', 'candle.n.01_1', 'basket.n.01_1'], ['inside', 'candle.n.01_2', 'basket.n.01_3'], ['inside', 'candle.n.01_3', 'basket.n.01_2'], ['inside', 'candle.n.01_4', 'basket.n.01_4']], [['inside', 'candle.n.01_1', 'basket.n.01_1'], ['inside', 'candle.n.01_2', 'basket.n.01_3'], ['inside', 'candle.n.01_3', 'basket.n.01_4'], ['inside', 'candle.n.01_4', 'basket.n.01_2']], [['inside', 'candle.n.01_1', 'basket.n.01_1'], ['inside', 'candle.n.01_2', 'basket.n.01_4'], ['inside', 'candle.n.01_3', 'basket.n.01_2'], ['inside', 'candle.n.01_4', 'basket.n.01_3']], [['inside', 'candle.n.01_1', 'basket.n.01_1'

TypeError: unhashable type: 'list'